# Convergence de PnP-ISTA avec débruiteurs MMSE

KHOUAJA Ahmed — M2 MIASHS, Lyon 2  
Cours Problèmes Inverses (N. Pustelnik)  
Article étudié : Xu, Sun, Liu, Wohlberg, Kamilov — *IEEE SPL*, 2020

---

## 1. Introduction

En traitement du signal, on est souvent confronté à la reconstruction d'un signal $\mathbf{x}$ à partir de mesures dégradées $\mathbf{y} = H\mathbf{x} + \mathbf{e}$, où $H$ modélise l'acquisition et $\mathbf{e}$ le bruit. Ce type de problème inverse est généralement mal posé et nécessite l'ajout d'un a priori sur $\mathbf{x}$ pour obtenir une solution raisonnable — c'est le principe de la régularisation, présenté dans les premiers chapitres du cours.

L'approche Plug-and-Play (PnP), introduite par Venkatakrishnan et al. en 2013, propose de remplacer le terme de régularisation par un débruiteur dans les itérations d'un algorithme proximal. L'article étudié ici prouve la convergence de PnP-ISTA lorsque le débruiteur est un estimateur MMSE. Les preuves antérieures supposaient la nonexpansivité du débruiteur, qui n'est pas vérifiée pour le MMSE. Les auteurs contournent cette difficulté via une approche de majorisation-minimisation.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

np.random.seed(2024)

## 2. Cadre théorique : ISTA et Plug-and-Play

La reconstruction régularisée revient à résoudre $\hat{\mathbf{x}} = \arg\min_{\mathbf{x}} g(\mathbf{x}) + h(\mathbf{x})$ avec $g(\mathbf{x}) = \frac{1}{2}\|\mathbf{y} - H\mathbf{x}\|^2$ le terme d'attache aux données et $h$ le régulariseur (norme $\ell_1$, variation totale, etc.).

L'algorithme ISTA, appelé aussi Forward-Backward en cours (chap. 4), itère :
$$z_t = x_{t-1} - \gamma \nabla g(x_{t-1}), \quad x_t = \mathrm{prox}_{\gamma h}(z_t)$$
avec $\gamma \leq 1/L$ et $L$ la constante de Lipschitz de $\nabla g$. L'opérateur proximal admet une interprétation bayésienne : si $h = -\log p_x$, le proximal correspond au débruiteur MAP pour un bruit gaussien de variance $\gamma$. C'est cette correspondance qui motive l'approche PnP, où on remplace directement le proximal par un débruiteur $D_\sigma$ :

$$z_t = x_{t-1} - \gamma \nabla g(x_{t-1}), \qquad x_t = D_\sigma(z_t)$$

La question est : est-ce que l'algorithme converge encore quand $D_\sigma$ n'est pas un opérateur proximal au sens strict ?

Il faut distinguer les débruiteurs MAP et MMSE. Le MAP cherche le mode de la distribution a posteriori et correspond au proximal d'un $h$ convexe — il est nonexpansif. Le MMSE calcule la moyenne a posteriori $\mathbb{E}[\mathbf{x}|\mathbf{z}]$ et minimise l'erreur quadratique, mais il est en général expansif. Or la plupart des débruiteurs utilisés en pratique (DnCNN entraîné en MSE, BM3D) sont des approximations du MMSE. Les preuves de convergence existantes (Chan 2017, Ryu 2019) ne s'y appliquent donc pas. La formule de Tweedie $D_\sigma(\mathbf{z}) = \mathbf{z} + \sigma^2 \nabla \log p_z(\mathbf{z})$ permet de montrer que le MMSE est en fait le proximal d'un $h_\mathrm{mmse}$ possiblement non convexe (via un résultat de Gribonval 2011).

## 3. Résultat de convergence

Sous trois hypothèses — prior non-dégénéré, gradient $L$-Lipschitz, et $f = g+h_\mathrm{mmse}$ bornée inférieurement — les auteurs prouvent que la suite $\{f(x_t)\}$ produite par PnP-ISTA avec le MMSE décroît de façon monotone et que $\|\nabla f(x_t)\| \to 0$. La preuve construit un majorant quadratique $\bar\mu(x,s) = g(s) + \langle \nabla g(s), x-s\rangle + \frac{1}{2\gamma}\|x-s\|^2 + h_\mathrm{mmse}(x)$ qui touche $f$ au point courant. Comme $D_\sigma = \mathrm{prox}_{\gamma h_\mathrm{mmse}}$, chaque itération minimise ce majorant, d'où la décroissance. Ce résultat ne nécessite ni la convexité de $h$ ni la nonexpansivité du débruiteur.

## 4. Résultats de l'article

L'évaluation porte sur un problème de compressive sensing avec un signal Bernoulli-Gaussien ($n=4096$, $\alpha = 0.1$), une matrice gaussienne i.i.d. et 20 dB de SNR d'entrée.

La convergence monotone de $f(x_t)/f(x_0)$ est confirmée. Le DnCNN entraîné en MSE reproduit quasi exactement le MMSE analytique, ce qui valide l'hypothèse que les réseaux entraînés en MSE sont des approximations MMSE. PnP-ISTA reste inférieur à GAMP car ISTA suppose un bruit AWGN dans les itérations, ce qui n'est pas vérifié en pratique.

## 5. Reproduction partielle

On reproduit le cadre de l'article en 1D (compressive sensing, débruiteur MMSE analytique) puis on étend au 2D (défloutage d'image avec DnCNN via DeepInverse). Les paramètres sont réduits pour le temps de calcul.

### 5.1 Compressive sensing avec MMSE exact

Pour un prior Bernoulli-Gaussien i.i.d., le débruiteur MMSE s'exprime analytiquement. Chaque composante est traitée indépendamment : $D_\sigma(z_i) = w(z_i) \frac{\sigma_x^2}{\sigma^2 + \sigma_x^2} z_i$ où $w(z_i) = P(x_i \neq 0 \mid z_i)$. Le paramètre $\sigma$ contrôle la force du débruitage.